### **INSTALLATIONS**

In [1]:
!pip install datasets

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
!pip install transformers

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
!pip install sentencepiece

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [4]:
!pip install sacremoses

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [5]:
!pip install importlib_metadata

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [6]:
!pip install nltk

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


### **IMPORT PACKAGES**

In [7]:
import nltk
import torch
import numpy as np
import pandas as pd
import torch.nn as nn
import torch.optim as optim
from nltk.stem.porter import *
import torch.utils.data as data
import torch.nn.functional as F
from datasets import load_dataset
from psutil import virtual_memory
from nltk.corpus import stopwords
from gensim.models import Word2Vec
from transformers import AutoConfig
from transformers import AutoTokenizer
from nltk.tokenize import word_tokenize
from sklearn.metrics import accuracy_score
from torch.nn.utils.rnn import pad_sequence
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from torch.utils.data import TensorDataset, DataLoader
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

### **CHECK MEMORY**

In [8]:
# Code from Colab -> was using to check memory
ram_gb = virtual_memory().total / 1e9
print('Your runtime has {:.1f} gigabytes of available RAM\n'.format(ram_gb))

if ram_gb < 20:
  print('Not using a high-RAM runtime')
else:
  print('You are using a high-RAM runtime!')

Your runtime has 54.8 gigabytes of available RAM

You are using a high-RAM runtime!


### **LOAD THE DATA**

In [9]:
dataset = load_dataset("yelp_review_full")

  0%|          | 0/2 [00:00<?, ?it/s]

### **LOAD AND SET UP THE TRANSFORMER MODEL**

In [10]:
dataFrame = pd.DataFrame(dataset["train"])
num_labels = len(dataFrame["label"].unique())

In [11]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

In [12]:
model_name = "bert-base-uncased"
model = torch.hub.load('huggingface/pytorch-transformers', 'model', model_name)
config = AutoConfig.from_pretrained(model_name)
last_state_dim = config.hidden_size
model.classifier = torch.nn.Linear(last_state_dim, num_labels)
model.to(device)
model.eval()

Using cache found in /root/.cache/torch/hub/huggingface_pytorch-transformers_main
Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(30522, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0-11): 12 x BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
  

### **DATA SPLITTING FOR TRANSFORMER MODEL**

In [13]:
train_dataset = dataset["train"]
test_dataset = dataset["test"]

In [14]:
chosen = range(100)
train_dataset = train_dataset.select(chosen)
test_dataset = test_dataset.select(chosen)

In [15]:
ratio = 0.8
size = len(train_dataset)
train_size = int(ratio * size)
val_size = size - train_size

In [16]:
train_dataset, val_dataset = torch.utils.data.random_split(train_dataset, [train_size, val_size])

In [17]:
train_loader = torch.utils.data.DataLoader(train_dataset, batch_size=16, shuffle=True)
val_loader = torch.utils.data.DataLoader(val_dataset, batch_size=16, shuffle=False)
test_loader = torch.utils.data.DataLoader(test_dataset, batch_size=16, shuffle=False)

### **PREPROCESSING FOR TRANSFORMER MODEL**

In [18]:
tokenizer = AutoTokenizer.from_pretrained(model_name)

In [19]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [20]:
def preprocess(text):
    
    stemmer = PorterStemmer()
    stop = set(stopwords.words('english'))

    index = 0
    for sentence in text:
      tokens = sentence.split()
      stemmed_tokens = [stemmer.stem(token) for token in tokens if token.lower() not in stop]
      new_sentence = ' '.join(stemmed_tokens)
      text[index] = new_sentence
      index += 1

    inputs = tokenizer(text, padding=True, truncation=True, return_tensors='pt')
    
    return inputs['input_ids'], inputs['attention_mask']

### **TRAINING**

In [21]:
def train_model(model, optimizer, epochs):

  model.train()

  criterion = nn.CrossEntropyLoss()

  train_predicts = []
  train_targets = []

  val_predicts = []
  val_targets = []

  for epoch in range(epochs):
      batch_idx = 0
      for batch in train_loader:

        label = batch['label']
        text = batch['text']

        input_ids, attention_mask = preprocess(text)
        input_ids, attention_mask, label = input_ids.to(device), attention_mask.to(device), label.to(device)

        optimizer.zero_grad()

        output = model(input_ids, attention_mask=attention_mask)
        last_hidden_states = output.last_hidden_state
        final_feature_map = last_hidden_states[:,-1,:]
        logits = model.classifier(final_feature_map)

        loss = criterion(logits, label)
        loss.backward()

        optimizer.step()

        batch_idx += 1

        predictions = torch.argmax(logits, dim=1)

        label = label.detach().cpu().numpy()
        predictions = torch.argmax(logits, dim=1).detach().cpu().numpy()

        train_predicts.extend(predictions)
        train_targets.extend(label)

        accuracy = accuracy_score(train_targets, train_predicts) * 100

        print("For Batch Number {bi} on Epoch Number {e} there is a Training Loss of {l} with a training accuracy of {a:.4f}%".format(bi=batch_idx, e=epoch, l=loss, a=accuracy))
         
      model.eval()

      with torch.no_grad(): 
        for batch in val_loader:
          
          label = batch['label']
          text = batch['text']

          input_ids, attention_mask = preprocess(text)
          input_ids, attention_mask, label = input_ids.to(device), attention_mask.to(device), label.to(device)

          output = model(input_ids, attention_mask=attention_mask)
          last_hidden_states = output.last_hidden_state
          final_feature_map = last_hidden_states[:,-1,:]
          logits = model.classifier(final_feature_map)

          loss = criterion(logits, label)

          predictions = torch.argmax(logits, dim=1)

          label = label.detach().cpu().numpy()
          predictions = torch.argmax(logits, dim=1).detach().cpu().numpy()

          val_predicts.extend(predictions)
          val_targets.extend(label)

          accuracy = accuracy_score(val_targets, val_predicts) * 100

          print("On Epoch Number {e} there is a Validation Loss of {l} and a validation accuracy of {a:.4f}%".format(e=epoch, l=loss, a=accuracy))

  train_accuracy = accuracy_score(train_targets, train_predicts) *  100
  train_precision = precision_score(train_targets, train_predicts, average='weighted', zero_division=1.0) * 100
  train_f1 = f1_score(train_targets, train_predicts, average='weighted') * 100
  train_recall = recall_score(train_targets, train_predicts, average='macro') * 100

  val_accuracy = accuracy_score(val_targets, val_predicts) * 100
  val_precision = precision_score(val_targets, val_predicts, average='weighted', zero_division=1.0) * 100
  val_f1 = f1_score(val_targets, val_predicts, average='weighted') * 100
  val_recall = recall_score(val_targets, val_predicts, average='macro') * 100

  print("TRAINING ACCURACY: {:.4f}%".format(train_accuracy))
  print("TRAINING PRECISION: {:.4f}%".format(train_precision))
  print("TRAINING F1: {:.4f}%".format(train_f1))
  print("TRAINING RECALL: {:.4f}%".format(train_recall))

  print("VALIDATION ACCURACY: {:.4f}%".format(val_accuracy))
  print("VALIDATION PRECISION: {:.4f}%".format(val_precision))
  print("VALIDATION F1: {:.4f}%".format(val_f1))
  print("VALIDATION RECALL: {:.4f}%".format(val_recall))

  print("..........................")

  print("Training is complete now.")

In [22]:
epochs = 10
learning_rate = 0.01
optimizer = torch.optim.Adam(model.classifier.parameters(), lr=learning_rate)

In [23]:
train_model(model, optimizer, epochs)

For Batch Number 1 on Epoch Number 0 there is a Training Loss of 1.6156675815582275 with a training accuracy of 18.7500%
For Batch Number 2 on Epoch Number 0 there is a Training Loss of 1.6775776147842407 with a training accuracy of 25.0000%
For Batch Number 3 on Epoch Number 0 there is a Training Loss of 2.0537309646606445 with a training accuracy of 18.7500%
For Batch Number 4 on Epoch Number 0 there is a Training Loss of 1.6675026416778564 with a training accuracy of 20.3125%
For Batch Number 5 on Epoch Number 0 there is a Training Loss of 1.6385350227355957 with a training accuracy of 21.2500%
On Epoch Number 0 there is a Validation Loss of 1.541701316833496 and a validation accuracy of 37.5000%
On Epoch Number 0 there is a Validation Loss of 1.3010544776916504 and a validation accuracy of 40.0000%
For Batch Number 1 on Epoch Number 1 there is a Training Loss of 1.3750563859939575 with a training accuracy of 25.0000%
For Batch Number 2 on Epoch Number 1 there is a Training Loss of 

### **TESTING**

In [24]:
def test_model(model):

  test_predicts = []
  test_targets = []

  batch_idx = 0
  for batch in test_loader:
    
    model.eval()

    with torch.no_grad():
      
      label = batch['label']  
      text = batch['text']

      input_ids, attention_mask = preprocess(text)
      input_ids, attention_mask, label = input_ids.to(device), attention_mask.to(device), label.to(device)

      output = model(input_ids, attention_mask=attention_mask)
      last_hidden_states = output.last_hidden_state
      final_feature_map = last_hidden_states[:,-1,:]
      logits = model.classifier(final_feature_map)

      predictions = torch.argmax(logits, dim=1)

      label = label.detach().cpu().numpy()
      predictions = torch.argmax(logits, dim=1).detach().cpu().numpy()
        
      test_predicts.extend(predictions)
      test_targets.extend(label)

      accuracy = accuracy_score(test_targets, test_predicts) * 100

      batch_idx += 1

      print("At Batch Number {bn} the loss is Testing Accuracy is {ta}%".format(bn=batch_idx, ta=accuracy))

  test_accuracy = accuracy_score(test_targets, test_predicts) * 100
  test_precision = precision_score(test_targets, test_predicts, average='weighted', zero_division=1.0) * 100
  test_f1 = f1_score(test_targets, test_predicts, average='weighted') * 100
  test_recall = recall_score(test_targets, test_predicts, average='macro') * 100

  print("TESTING ACCURACY: {:.4f}%".format(test_accuracy))
  print("TESTING PRECISION: {:.4f}%".format(test_precision))
  print("TESTING F1: {:.4f}%".format(test_f1))
  print("TESTING RECALL: {:.4f}%".format(test_recall))

  print("..........................")

  print("Testing is complete now.")

In [25]:
test_model(model)

At Batch Number 1 the loss is Testing Accuracy is 43.75%
At Batch Number 2 the loss is Testing Accuracy is 31.25%
At Batch Number 3 the loss is Testing Accuracy is 35.41666666666667%
At Batch Number 4 the loss is Testing Accuracy is 31.25%
At Batch Number 5 the loss is Testing Accuracy is 32.5%
At Batch Number 6 the loss is Testing Accuracy is 33.33333333333333%
At Batch Number 7 the loss is Testing Accuracy is 33.0%
TESTING ACCURACY: 33.0000%
TESTING PRECISION: 36.4251%
TESTING F1: 32.6460%
TESTING RECALL: 32.0401%
..........................
Testing is complete now.


### **DATA SPLITTING FOR REGRESSION**

In [26]:
X_train = [data["text"] for data in train_dataset]
Y_train = [data["label"] for data in train_dataset]

X_test = [data["text"] for data in test_dataset]
Y_test = [data["label"] for data in test_dataset]

### **PREPROCESSING FOR REGRESSION**

In [27]:
  vectorizer = TfidfVectorizer()

In [28]:
  X_train_transform = vectorizer.fit_transform(X_train)
  X_test_transform = vectorizer.transform(X_test)

### **LOGISTIC REGRESSION**

In [29]:
LR_model = LogisticRegression(penalty='l2',max_iter=1000, C=0.5, class_weight='balanced')
LR_model.fit(X_train_transform, Y_train)

LogisticRegression(C=0.5, class_weight='balanced', max_iter=1000)

In [30]:
Y_prediction = LR_model.predict(X_test_transform)

In [31]:
reg_accuracy = accuracy_score(Y_test, Y_prediction) * 100
reg_precision = precision_score(Y_test, Y_prediction, average='weighted') * 100
reg_f1_score = f1_score(Y_test, Y_prediction, average='weighted') * 100
reg_recall = recall_score(Y_test, Y_prediction, average='macro') * 100

In [32]:
print("Logistic Regression Accuracy: {:.4f}%".format(reg_accuracy))
print("Logistic Regression Precision: {:.4f}%".format(reg_precision))
print("Logistic Regression F1 Score: {:.4f}%".format(reg_f1_score))
print("Logistic Regression Recall: {:.4f}%".format(reg_recall))

Logistic Regression Accuracy: 35.0000%
Logistic Regression Precision: 34.5332%
Logistic Regression F1 Score: 33.2330%
Logistic Regression Recall: 32.7361%


### **CITATIONS**


*   Professor's Code
*   TA's Code
*   "Pandas.Unique#.” Pandas.Unique - Pandas 2.0.2 Documentation, 2023, pandas.pydata.org/docs/reference/api/pandas.unique.html. 
*   Process, huggingface.co/docs/datasets/process. 
*   “Python Torch.Utils.Data.Random_split() Examples.” Python Examples of Torch.Utils.Data.Random_split, www.programcreek.com/python/example/125046/torch.utils.data.random_split. 
*   “Documentation.” NLTK, 2 Jan. 2023, www.nltk.org/howto/stem.html. 
*   “Removing Stop Words with NLTK in Python.” GeeksforGeeks, 16 May 2023, www.geeksforgeeks.org/removing-stop-words-nltk-python/. 
*   “Torch.Argmax¶.” Torch.Argmax - PyTorch 2.0 Documentation, 2023, pytorch.org/docs/stable/generated/torch.argmax.html. 
*   Admin. “Create Numpy Array from Pytorch Tensor Using Detach().Numpy().” For Machine Learning, 5 May 2023, androidkt.com/create-numpy-array-from-pytorch-tensor-using-detach-numpy/. 
*  “3.3. Metrics and Scoring: Quantifying the Quality of Predictions.” Scikit, 2007, scikit-learn.org/stable/modules/model_evaluation.html.
*   “Sklearn.Feature_extraction.Text.TfidfVectorizer.” Scikit, 2007, scikit-learn.org/stable/modules/generated/sklearn.feature_extraction.text.TfidfVectorizer.html. 
*   














